In [21]:
import h5py
import hdf5plugin
import io
import numpy as np
import xml.etree.ElementTree as ET

In [22]:
hdf5 = h5py.File('../resources/small.mzMLb', 'r+')

In [23]:
hdf5.keys()

<KeysViewHDF5 ['chromatogram_MS_1000515_float32', 'chromatogram_MS_1000595_float32', 'mzML', 'mzML_chromatogramIndex', 'mzML_chromatogramIndex_idRef', 'mzML_spectrumIndex', 'mzML_spectrumIndex_idRef', 'spectrum_MS_1000514_float64', 'spectrum_MS_1000515_float32']>

In [24]:
hdf5['mzML']

<HDF5 dataset "mzML": shape (287267,), type "|i1">

In [25]:
class HDF5ByteBuffer(io.RawIOBase):
    def __init__(self, buffer, offset=None):
        if offset is None:
            offset = 0
        self.buffer = buffer
        self.offset = offset
        self.size = self.buffer.size
        self.mode = 'rb'

    def readable(self):
        return True

    def seekable(self):
        return True

    def isatty(self):
        return False

    def seek(self, offset, whence=0):
        if whence == io.SEEK_SET:
            self.offset = offset
        elif whence == io.SEEK_CUR:
            self.offset += offset
        elif whence == io.SEEK_END:
            self.offset = self.size - offset
        else:
            raise ValueError(f"Bad whence {whence}")
        return self.offset

    def tell(self):
        return self.offset

    def close(self):
        return

    @property
    def closed(self):
        return False

    def readinto(self, b):
        n = len(b)
        temp = self._read(n)
        m = len(temp)
        b[:m] = temp[:]
        return m

    def readall(self):
        return bytes(self._read(-1))

    def read(self, n=-1):
        return bytes(self._read(n))

    def write(self, b):
        raise ValueError("Read-only stream")

    def _read(self, n=-1):
        if n == -1:
            n = self.size + 1
        dat = bytearray(np.array(self.buffer[self.offset:self.offset + n]))
        self.offset += n
        return dat


In [26]:
mzxml = io.BufferedReader(HDF5ByteBuffer(hdf5['mzML']))
xml_bytes = mzxml.read()

In [27]:
tree = ET.ElementTree(ET.fromstring(xml_bytes))
tree.getroot()

<Element '{http://psi.hupo.org/ms/mzml}mzML' at 0x7fb2732e24f0>